# Server-IP

Die nachfolgende Zeile ermittelt die öffentliche IP-Adresse eines Servers, der in einer Cloud-Umgebung betrieben wird, und speichert diese in einer Datei. 

Das Skript unterstützt verschiedene Cloud-Anbieter wie AWS, Azure und MAAS. 

Abhängig vom erkannten Cloud-Anbieter wird die jeweilige Methode zur Abfrage der öffentlichen IP-Adresse genutzt. Sollte der Cloud-Anbieter nicht erkannt werden, wird die IP-Adresse basierend auf der Netzwerkschnittstelle des Servers ermittelt. 

Die ermittelte IP-Adresse wird dann in der Datei server-ip im Verzeichnis ~/work/ gespeichert, wo sie die anderen Notebooks wieder ansprechen.

In [ ]:
%%bash
# Public IP anhand Cloud Provider setzen, WireGuard ueberschreibt alle
cloud_provider=$(cloud-init query v1.cloud_name 2>/dev/null) 
case "$cloud_provider" in
      "aws")
        public_ip=$(sudo cloud-init query ds.meta_data.public_hostname)
        ;;
      "gce" | "gcloud")
        public_ip=$(curl -s "http://metadata.google.internal/computeMetadata/v1/instance/network-interfaces/0/access-configs/0/external-ip" -H "Metadata-Flavor: Google")
        ;;  
    "azure")
        public_ip=$(jq -r '.ds.meta_data.imds.network.interface[0].ipv4.ipAddress[0].publicIpAddress' /run/cloud-init/instance-data.json 2>/dev/null)
        ;;
    "maas")
        public_ip=$(hostname).maas
        ;;        
    *)
        public_ip=$(hostname -I | cut -d ' ' -f 1) 
        ;;
esac
echo $public_ip >~/work/server-ip

# aktivieren wenn mit WireGuard gearbeitet wird
# wg_ip=$(ip -f inet addr show wg0 2>/dev/null | grep -Po 'inet \K[\d.]+') 
# [ "$wg_ip" != "" ] && { echo $wg_ip >~/work/server-ip; }

echo $public_ip

- - -

# UUID

Die nachfolgende Zeile generiert eine eindeutige UUID (Universally Unique Identifier).

Die UUID dient als unverwechselbare Kennung, die sicherstellt, dass jede Installation eindeutig identifiziert werden kann. 

Damit kann, z.B. für die IoT Übungen ein eindeutiges MQTT Topic bereitgestellt werden.

Die generierte UUID wird im Python-Dateiformat uuid.py im Verzeichnis ~/work/ gespeichert und wird in den Python-Skripten weiterverwendet.

In [ ]:
! echo "UUID=\"$(uuid)\"" >~/work/uuid.py